# Laboratório 3.7 - Notebook do aluno

## Visão geral

Este laboratório é uma continuação dos laboratórios guiados do Módulo 3.

Neste laboratório, você criará um trabalho de ajuste de hiperparâmetros para ajustar o modelo criado anteriormente. Depois, você comparará as métricas dos dois modelos.


## Introdução ao cenário do negócio

Você trabalha para um provedor de serviços médicos e deseja melhorar a detecção de anormalidades em pacientes ortopédicos. 

Você tem a incumbência de resolver esse problema usando machine learning (ML). Você tem acesso a um conjunto de dados que contém seis funcionalidades biomecânicas (componentes) e um alvo (target) de *normal* (normal) ou *abnormal* (anormal). Você pode usar esse conjunto de dados (dataset) para treinar um modelo de ML para prever se um paciente terá uma anomalia.


## Sobre esse conjunto de dados (dataset)

Esse conjunto de dados biomédicos foi criado pelo Dr. Henrique da Mota durante um período de residência médica no Group of Applied Research in Orthopaedics (GARO) do Centre Médico-Chirurgical de Réadaptation des Massues em Lyon, na França. Os dados foram organizados em duas tarefas de classificação diferentes, mas relacionadas. 

A primeira tarefa consiste em classificar os pacientes como pertencentes a uma das três categorias a seguir: 

- *Normal* (Normal) (100 pacientes)
- *Disk Hernia* (Hérnia de disco) (60 pacientes)
- *Spondylolisthesis* (Espondilolistese) (150 pacientes)

Para a segunda tarefa, as categorias *Disk Hernia* (Hérnia de disco) e *Spondylolisthesis* (Espondilolistese) foram mescladas em uma única categoria, rotulada como *abnormal* (anormal). Portanto, a segunda tarefa consiste em classificar os pacientes como pertencentes a uma das categorias: *Normal* (Normal) (100 pacientes) ou *Abnormal* (Anormal) (210 pacientes).


## Informações dos atributos

Cada paciente é representado no conjunto de dados (dataset) por seis atributos biomecânicos derivados da forma e da orientação da pelve e da coluna lombar (nesta ordem): 

- Incidência pélvica
- Inclinação pélvica
- Ângulo da lordose lombar
- Inclinação sacral
- Raio pélvico
- Grau de espondilolistese

A convenção a seguir é usada para os rótulos de classe (labels): 
- DH (hérnia de disco)
- Espondilolistese (SL)
- Normal (NO) 
- Anormal (AB)


Para obter mais informações sobre esse conjunto de dados (dataset), consulte a [página da Web Conjunto de dados de coluna vertebral](http://archive.ics.uci.edu/ml/datasets/Vertebral+Column).


## Atribuições do conjunto de dados (dataset)

Esse conjunto de dados (dataset) foi obtido de:
Dua, D. e Graff, C. (2019). repositório UCI Machine Learning (http://archive.ics.uci.edu/ml). Irvine, CA: University of California, School of Information and Computer Science.


# Configuração do laboratório

Como a solução é dividida em vários laboratórios no módulo, você executará as seguintes células para poder carregar os dados e treinar o modelo a ser implantado.

**Observação:** a configuração pode demorar até cinco minutos para ser concluída.

## Importação dos dados, treinamento, teste e validação do modelo

Ao executar as seguintes células, os dados serão importados, e o modelo será treinado, testado e validado. Ele estará pronto para uso. 

**Observação:** as células a seguir representam as principais etapas dos laboratórios anteriores.

Para ajustar o modelo, ele deve estar pronto. Você poderá então ajustá-lo com hiperparâmetros na etapa 2.

In [ ]:
bucket='c117523a2804664l6884813t1w571208947195-labbucket-1yte2gh3rtlw'

In [ ]:
import time
start = time.time()
import warnings, requests, zipfile, io
warnings.simplefilter('ignore')
import pandas as pd
from scipy.io import arff

import os
import boto3
import sagemaker
from sagemaker.image_uris import retrieve
from sklearn.model_selection import train_test_split

from sklearn.metrics import roc_auc_score, roc_curve, auc, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

**Observação:** a célula leva, aproximadamente, **10** minutos para ser executada. Observe o código e como ele é processado. Isso ajudará você a entender melhor o que está acontecendo em segundo plano. Lembre-se de que esta célula conclui todas as etapas que você realizou em laboratórios anteriores neste módulo, incluindo:
 - Importar os dados
 - Carregar os dados em um dataframe
 - Dividir os dados em conjuntos de dados de treinamento, teste e validação
 - Fazer upload dos conjuntos de dados divididos no S3
 - Treinar, testar e validar o modelo com os conjuntos de dados

In [ ]:
%%time

def plot_roc(test_labels, target_predicted_binary):
    TN, FP, FN, TP = confusion_matrix(test_labels, target_predicted_binary).ravel()
    # Sensitivity, hit rate, recall, or true positive rate
    Sensitivity  = float(TP)/(TP+FN)*100
    # Specificity or true negative rate
    Specificity  = float(TN)/(TN+FP)*100
    # Precision or positive predictive value
    Precision = float(TP)/(TP+FP)*100
    # Negative predictive value
    NPV = float(TN)/(TN+FN)*100
    # Fall out or false positive rate
    FPR = float(FP)/(FP+TN)*100
    # False negative rate
    FNR = float(FN)/(TP+FN)*100
    # False discovery rate
    FDR = float(FP)/(TP+FP)*100
    # Overall accuracy
    ACC = float(TP+TN)/(TP+FP+FN+TN)*100

    print(f"Sensitivity or TPR: {Sensitivity}%")    
    print(f"Specificity or TNR: {Specificity}%") 
    print(f"Precision: {Precision}%")   
    print(f"Negative Predictive Value: {NPV}%")  
    print( f"False Positive Rate: {FPR}%") 
    print(f"False Negative Rate: {FNR}%")  
    print(f"False Discovery Rate: {FDR}%" )
    print(f"Accuracy: {ACC}%") 

    test_labels = test.iloc[:,0];
    print("Validation AUC", roc_auc_score(test_labels, target_predicted_binary) )

    fpr, tpr, thresholds = roc_curve(test_labels, target_predicted_binary)
    roc_auc = auc(fpr, tpr)

    plt.figure()
    plt.plot(fpr, tpr, label='ROC curve (area = %0.2f)' % (roc_auc))
    plt.plot([0, 1], [0, 1], 'k--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver operating characteristic')
    plt.legend(loc="lower right")

    # create the axis of thresholds (scores)
    ax2 = plt.gca().twinx()
    ax2.plot(fpr, thresholds, markeredgecolor='r',linestyle='dashed', color='r')
    ax2.set_ylabel('Threshold',color='r')
    ax2.set_ylim([thresholds[-1],thresholds[0]])
    ax2.set_xlim([fpr[0],fpr[-1]])

    print(plt.figure())

def plot_confusion_matrix(test_labels, target_predicted):
    matrix = confusion_matrix(test_labels, target_predicted)
    df_confusion = pd.DataFrame(matrix)
    colormap = sns.color_palette("BrBG", 10)
    sns.heatmap(df_confusion, annot=True, fmt='.2f', cbar=None, cmap=colormap)
    plt.title("Confusion Matrix")
    plt.tight_layout()
    plt.ylabel("True Class")
    plt.xlabel("Predicted Class")
    plt.show()
    

f_zip = 'http://archive.ics.uci.edu/ml/machine-learning-databases/00212/vertebral_column_data.zip'
r = requests.get(f_zip, stream=True)
Vertebral_zip = zipfile.ZipFile(io.BytesIO(r.content))
Vertebral_zip.extractall()

data = arff.loadarff('column_2C_weka.arff')
df = pd.DataFrame(data[0])

class_mapper = {b'Abnormal':1,b'Normal':0}
df['class']=df['class'].replace(class_mapper)

cols = df.columns.tolist()
cols = cols[-1:] + cols[:-1]
df = df[cols]

train, test_and_validate = train_test_split(df, test_size=0.2, random_state=42, stratify=df['class'])
test, validate = train_test_split(test_and_validate, test_size=0.5, random_state=42, stratify=test_and_validate['class'])

prefix='lab3'

train_file='vertebral_train.csv'
test_file='vertebral_test.csv'
validate_file='vertebral_validate.csv'

s3_resource = boto3.Session().resource('s3')
def upload_s3_csv(filename, folder, dataframe):
    csv_buffer = io.StringIO()
    dataframe.to_csv(csv_buffer, header=False, index=False )
    s3_resource.Bucket(bucket).Object(os.path.join(prefix, folder, filename)).put(Body=csv_buffer.getvalue())

upload_s3_csv(train_file, 'train', train)
upload_s3_csv(test_file, 'test', test)
upload_s3_csv(validate_file, 'validate', validate)

container = retrieve('xgboost',boto3.Session().region_name,'1.0-1')

hyperparams={"num_round":"42",
             "eval_metric": "auc",
             "objective": "binary:logistic",
             "silent" : 1}

s3_output_location="s3://{}/{}/output/".format(bucket,prefix)
xgb_model=sagemaker.estimator.Estimator(container,
                                       sagemaker.get_execution_role(),
                                       instance_count=1,
                                       instance_type='ml.m5.2xlarge',
                                       output_path=s3_output_location,
                                        hyperparameters=hyperparams,
                                        sagemaker_session=sagemaker.Session())

train_channel = sagemaker.inputs.TrainingInput(
    "s3://{}/{}/train/".format(bucket,prefix,train_file),
    content_type='text/csv')

validate_channel = sagemaker.inputs.TrainingInput(
    "s3://{}/{}/validate/".format(bucket,prefix,validate_file),
    content_type='text/csv')

data_channels = {'train': train_channel, 'validation': validate_channel}

xgb_model.fit(inputs=data_channels, logs=False)

batch_X = test.iloc[:,1:];

batch_X_file='batch-in.csv'
upload_s3_csv(batch_X_file, 'batch-in', batch_X)

batch_output = "s3://{}/{}/batch-out/".format(bucket,prefix)
batch_input = "s3://{}/{}/batch-in/{}".format(bucket,prefix,batch_X_file)

xgb_transformer = xgb_model.transformer(instance_count=1,
                                       instance_type='ml.m5.2xlarge',
                                       strategy='MultiRecord',
                                       assemble_with='Line',
                                       output_path=batch_output)

xgb_transformer.transform(data=batch_input,
                         data_type='S3Prefix',
                         content_type='text/csv',
                         split_type='Line')
xgb_transformer.wait(logs=False)

# Etapa 1: Obtenção das estatísticas do modelo

Antes de ajustar o modelo, familiarize-se novamente com as métricas do modelo atual.

A configuração executou uma previsão em lote. Portanto, você deve ler os resultados do Amazon Simple Storage Service (Amazon S3).

In [ ]:
s3 = boto3.client('s3')
obj = s3.get_object(Bucket=bucket, Key="{}/batch-out/{}".format(prefix,'batch-in.csv.out'))
target_predicted = pd.read_csv(io.BytesIO(obj['Body'].read()),names=['class'])

def binary_convert(x):
    threshold = 0.5
    if x > threshold:
        return 1
    else:
        return 0

target_predicted_binary = target_predicted['class'].apply(binary_convert)
test_labels = test.iloc[:,0]



Trace a matriz de confusão (confusion matrix) e a curva da característica de operação do receptor (ROC) para o modelo original.


In [ ]:
plot_confusion_matrix(test_labels, target_predicted_binary)

In [ ]:
plot_roc(test_labels, target_predicted_binary)

Esse gráfico fornece um ponto de partida. Faça uma anotação da *Área de validação sob a curva (AUC)*. Você a usará mais tarde para verificar se seu modelo ajustado é melhor. 

# Etapa 2: Criar um trabalho de ajuste de hiperparâmetros

Um trabalho de ajuste de hiperparâmetros pode demorar várias horas para ser concluído, dependendo dos intervalos de valores que você fornecer. Para simplificar essa tarefa, os parâmetros usados nesta etapa são um subconjunto dos intervalos recomendados. Eles foram ajustados para fornecer bons resultados neste laboratório, sem levar várias horas para serem concluídos.

Para obter mais informações sobre os parâmetros a serem ajustados para XGBoost, consulte [Ajustar um modelo XGBoost](https://docs.aws.amazon.com/sagemaker/latest/dg/xgboost-tuning.html) na documentação da AWS.

Como essa próxima célula pode levar cerca de **45** minutos para ser executada, prossiga e execute a célula. Você examinará o que está ocorrendo e por que esses intervalos de hiperparâmetros foram escolhidos.

In [ ]:
%%time
from sagemaker.tuner import IntegerParameter, CategoricalParameter, ContinuousParameter, HyperparameterTuner

xgb = sagemaker.estimator.Estimator(container,
                                    role=sagemaker.get_execution_role(), 
                                    instance_count= 1, # make sure you have limit set for these instances
                                    instance_type='ml.m4.xlarge', 
                                    output_path='s3://{}/{}/output'.format(bucket, prefix),
                                    sagemaker_session=sagemaker.Session())


xgb.set_hyperparameters(eval_metric='error@.40',
                        objective='binary:logistic',
                        num_round=42)

hyperparameter_ranges = {'alpha': ContinuousParameter(0, 100),
                         'min_child_weight': ContinuousParameter(1, 5),
                         'subsample': ContinuousParameter(0.5, 1),
                         'eta': ContinuousParameter(0.1, 0.3),  
                         'num_round': IntegerParameter(1,50)
                         }

objective_metric_name = 'validation:error'
objective_type = 'Minimize'

tuner = HyperparameterTuner(xgb,
                            objective_metric_name,
                            hyperparameter_ranges,
                            max_jobs=10, # Set this to 10 or above depending upon budget & available time.
                            max_parallel_jobs=1,
                            objective_type=objective_type,
                            early_stopping_type='Auto')

tuner.fit(inputs=data_channels, include_cls_metadata=False)
tuner.wait()

Primeiramente, você criará o modelo que deseja ajustar.

```
xgb = sagemaker.estimator.Estimator(container,
                                   role=sagemaker.get_execution_role(), 
                                   instance_count= 1, # verifique se há limite definido para essas instâncias
                                   instance_type='ml.m4.xlarge', 
                                   output_path='s3://{}/{}/output'.format(bucket, prefix),
                                   sagemaker_session=sagemaker.Session())

xgb.set_hyperparameters(eval_metric='[error@.40]',
                        objective='binary:logistic',
                        num_round=42)

```                        

Observe que a *eval_metric* do modelo foi alterada para *error@.40*, com o objetivo de minimizar esse valor. 

**error** é a taxa de erro da classificação binária. Ela é calculada como *#(wrong cases)/#(all cases)* (n.º de casos incorretos)/(n.º total de casos). Para predições, a avaliação considerará as instâncias que têm um valor de predição maior do que 0,4 como instâncias positivas e as outras como instâncias negativas.

Em seguida, você deverá especificar os hiperparâmetros que deseja ajustar, além dos intervalos que você deverá selecionar para cada parâmetro.

Os hiperparâmetros que têm o maior efeito nas métricas objetivas do XGBoost são: 

- alpha
- min_child_weight
- subsample
- eta
- num_round 

Os intervalos de ajuste (tuning ranges) recomendados podem ser encontrados na documentação da AWS em [Ajustando um modelo XGBoost](https://docs.aws.amazon.com/sagemaker/latest/dg/xgboost-tuning.html).

Para este laboratório, você usará um *subconjunto* de valores. Esses valores foram obtidos executando o trabalho de ajuste (tuning job) com o intervalo completo e, em seguida, minimizando o intervalo para que você possa usar menos iterações para obter melhor desempenho. Embora essa prática não seja estritamente realista, ela impede que você espere várias horas neste laboratório até que o trabalho de ajuste seja concluído.

```
hyperparameter_ranges = {'alpha': ContinuousParameter(0, 100),
                         'min_child_weight': ContinuousParameter(1, 5),
                         'subsample': ContinuousParameter(0.5, 1),
                         'eta': ContinuousParameter(0.1, 0.3),  
                         'num_round': IntegerParameter(1,50)
                         }
```


Você deve especificar como está classificando o modelo. Você pode usar várias métricas objetivas diferentes, sendo que um subconjunto se aplica a um problema de classificação binária. Como a métrica de avaliação é **error**, você define o objetivo como *error*.

```
objective_metric_name = 'validation:error'
objective_type = 'Minimize'
```

Por fim, você executa o trabalho de ajuste.

```
tuner = HyperparameterTuner(xgb,
                            objective_metric_name,
                            hyperparameter_ranges,
                            max_jobs=10, # Defina isso como 10 ou acima dependendo do orçamento e do tempo disponível.
                            max_parallel_jobs=1,
                            objective_type=objective_type,
                            early_stopping_type='Auto')

tuner.fit(inputs=data_channels, include_cls_metadata=False)
tuner.wait()
```



<i class="fas fa-exclamation-triangle" style="color:red"></i> Aguarde até que o trabalho de treinamento seja concluído. Isso pode levar até **45** minutos. Enquanto você aguarda, observe o status do trabalho no console, conforme descrito nas instruções a seguir.

**Para monitorar trabalhos de otimização de hiperparâmetros:**  

1. No Console de gerenciamento da AWS, no menu **Services** (Serviços), escolha **Amazon SageMaker**.  
2. Escolha **Training > Hyperparameter tuning jobs** (Treinamento > Trabalhos de ajuste de hiperparâmetros).  
3. Você pode verificar o status de cada trabalho de ajuste de hiperparâmetros, seu valor de métrica objetiva e seus logs.  

Depois que o trabalho de treinamento for concluído, verifique o trabalho e certifique-se de que ele tenha sido concluído com êxito.

In [ ]:
boto3.client('sagemaker').describe_hyper_parameter_tuning_job(
    HyperParameterTuningJobName=tuner.latest_tuning_job.job_name)['HyperParameterTuningJobStatus']

# Etapa 3: Investigação dos resultados do trabalho de ajuste (tuning jobs)

Agora que o trabalho está concluído, deve haver 10 trabalhos (jobs) concluídos. Um dos trabalhos (jobs) deve ser marcado como o me.lhor.

Você pode examinar as métricas obtendo *HyperparameterTuningJobAnalytics* e carregando esses dados em um DataFrame do pandas.


In [ ]:
from pprint import pprint
from sagemaker.analytics import HyperparameterTuningJobAnalytics

tuner_analytics = HyperparameterTuningJobAnalytics(tuner.latest_tuning_job.name, sagemaker_session=sagemaker.Session())

df_tuning_job_analytics = tuner_analytics.dataframe()

# Sort the tuning job analytics by the final metrics value
df_tuning_job_analytics.sort_values(
    by=['FinalObjectiveValue'],
    inplace=True,
    ascending=False if tuner.objective_type == "Maximize" else True)

# Show detailed analytics for the top 20 models
df_tuning_job_analytics.head(20)


Você deve ser capaz de ver os hiperparâmetros que foram usados para cada job, juntamente com a pontuação. Você pode usar esses parâmetros e criar um modelo ou pode obter o melhor modelo do trabalho de ajuste de hiperparâmetros (hyperparameter tuning job).

In [ ]:
attached_tuner = HyperparameterTuner.attach(tuner.latest_tuning_job.name, sagemaker_session=sagemaker.Session())
best_training_job = attached_tuner.best_training_job()

Agora, você deve usar o melhor trabalho de treinamento e criar o modelo.

In [ ]:
from sagemaker.estimator import Estimator
algo_estimator = Estimator.attach(best_training_job)

best_algo_model = algo_estimator.create_model(env={'SAGEMAKER_DEFAULT_INVOCATIONS_ACCEPT':"text/csv"})

Em seguida, você pode usar o método de transformação (transform) para executar uma predição em lote usando seus dados de teste. Lembre-se de que os dados de teste são dados que o modelo nunca viu antes.

In [ ]:
%%time
batch_output = "s3://{}/{}/batch-out/".format(bucket,prefix)
batch_input = "s3://{}/{}/batch-in/{}".format(bucket,prefix,batch_X_file)

xgb_transformer = best_algo_model.transformer(instance_count=1,
                                       instance_type='ml.m4.xlarge',
                                       strategy='MultiRecord',
                                       assemble_with='Line',
                                       output_path=batch_output)


xgb_transformer.transform(data=batch_input,
                         data_type='S3Prefix',
                         content_type='text/csv',
                         split_type='Line')
xgb_transformer.wait(logs=False)

Obtenha o alvo predito e os rótulos de teste do modelo.

In [ ]:
s3 = boto3.client('s3')
obj = s3.get_object(Bucket=bucket, Key="{}/batch-out/{}".format(prefix,'batch-in.csv.out'))
best_target_predicted = pd.read_csv(io.BytesIO(obj['Body'].read()),names=['class'])

def binary_convert(x):
    threshold = 0.5
    if x > threshold:
        return 1
    else:
        return 0

best_target_predicted_binary = best_target_predicted['class'].apply(binary_convert)
test_labels = test.iloc[:,0]

Plote uma matriz de confusão para `best_target_predicted` e `test_labels`.

In [ ]:
plot_confusion_matrix(test_labels, best_target_predicted_binary)

Plote o gráfico ROC.

In [ ]:
plot_roc(test_labels, best_target_predicted_binary)

**Pergunta:** Como esses resultados diferem do original? Esses resultados são melhores ou piores? 

Talvez você nem sempre veja uma melhoria. Há alguns motivos para esse resultado:

- O modelo já pode ser bom desde a passagem inicial (o que conta como *bom* é subjetivo).
- Você não tem uma grande quantidade de dados para treinar.
- Você está usando um *subconjunto* dos intervalos de ajuste de hiperparâmetros para economizar tempo neste laboratório.

Aumentar os intervalos de hiperparâmetros (conforme recomendado pela documentação) e executar mais de 30 trabalhos (jobs) normalmente melhorará o modelo. No entanto, esse processo demorará de 2 a 3 horas para ser concluído.

# Parabéns!

Você concluiu este laboratório e agora pode encerrá-lo seguindo as instruções do guia do laboratório.


